In [6]:
from split_environment import BlackjackEnv
from basic_strategy_agent import BasicStrategyAgent
from split_train_agent import evaluate_agent
from Supervised_betting import collect_dictionary
from Supervised_betting import evaluate_nn_betting_agent
from BettingModel import train_betting_nn
from HiloAgent import HiLoBettingAgent

count_type = "hi_lo"
env = BlackjackEnv(count_type = "full")
agent = BasicStrategyAgent()

#model = HiLoBettingAgent(base_bet=1, max_units=100)

dictionary = collect_dictionary(agent, env, count_type, 10000)
print(dictionary)

#model = train_betting_nn(dictionary, count_type, batch_size=64, epochs=1000)
#evaluate_nn_betting_agent(model, agent, env, count_type=count_type, episodes = 100000, max_bet = 100, min_bet = 1)

Episode 1000/10000 — Last Reward: -1.00 — Count State: (1.0,)
Episode 2000/10000 — Last Reward: 1.00 — Count State: (-2.0,)
Episode 3000/10000 — Last Reward: 1.00 — Count State: (-1.0,)
Episode 4000/10000 — Last Reward: 1.00 — Count State: (2.0,)
Episode 5000/10000 — Last Reward: 1.00 — Count State: (-0.0,)
Episode 6000/10000 — Last Reward: 1.00 — Count State: (1.0,)
Episode 7000/10000 — Last Reward: 1.00 — Count State: (2.0,)
Episode 8000/10000 — Last Reward: -1.00 — Count State: (-1.0,)
Episode 9000/10000 — Last Reward: -1.00 — Count State: (0.0,)
Episode 10000/10000 — Last Reward: -1.00 — Count State: (-1.0,)
{(-0.0,): [0.35196850393700874, 0.5173228346456676, 0.13070866141732296, 2540.0], (-1.0,): [0.38062500000000044, 0.466875, 0.15249999999999977, 1600.0], (1.0,): [0.3700787401574803, 0.5118110236220463, 0.11811023622047244, 1651.0], (2.0,): [0.36244019138756006, 0.503588516746412, 0.1339712918660288, 836.0], (-2.0,): [0.39075630252100807, 0.45588235294117646, 0.15336134453781528

In [5]:
model = train_betting_nn(dictionary, count_type, batch_size=64, epochs=10000)
evaluate_nn_betting_agent(model, agent, env, count_type=count_type, episodes = 10000, max_bet = 100, min_bet = 1)

Epoch 1/10000, Loss: 1.0630
Epoch 1001/10000, Loss: 1.0029
Epoch 2001/10000, Loss: 0.9970
Epoch 3001/10000, Loss: 0.9924
Epoch 4001/10000, Loss: 0.9910
Epoch 5001/10000, Loss: 0.9904
Epoch 6001/10000, Loss: 0.9901
Epoch 7001/10000, Loss: 0.9899
Epoch 8001/10000, Loss: 0.9899
Epoch 9001/10000, Loss: 0.9899

Evaluation over 10000 episodes:
Average profit per hand: -0.1173
Win rate: 0.3651
Loss rate: 0.4956
Push rate: 0.1393


-0.1173

In [2]:
from split_environment import BlackjackEnv
from basic_strategy_agent import BasicStrategyAgent
from split_train_agent import evaluate_agent
from Supervised_betting import collect_dictionary
from Supervised_betting import evaluate_nn_betting_agent
from BettingModel import train_betting_nn
from HiloAgent import HiLoBettingAgent
from basic_strategy_agent import compare_with_basic_strategy

count_type = "hi_lo"
env = BlackjackEnv(count_type = "full")
agent = BasicStrategyAgent()

evaluate_agent(agent, env, episodes=10000)

ValueError: SOMETHING ILLEGAL HAS HAPPENED. Action = 2